# Make EIS Fe XII 195.119 Å Movie
Make a movie of an EIS Fe XII 195 Å emission map for 5000 s. Just to show off what you can do with ChiantiPy.

In [1]:
import os

from scipy.interpolate import interp1d,splev,splrep
from scipy.ndimage import gaussian_filter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import astropy.units as u
import astropy.constants as const
from sunpy.map import Map

from synthesizAR.instruments import InstrumentHinodeEIS,InstrumentSDOAIA
from synthesizAR.util import EISCube
from synthesizAR.atomic import EmissionModel

%matplotlib inline

/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


 using cli
 using CLI for selections
 reading chiantirc file


In [2]:
eis = InstrumentHinodeEIS([0,2.2e4]*u.s)

In [3]:
emiss_model = EmissionModel.restore('/data/datadrive1/ar_forward_modeling/systematic_ar_study/emission_model1109_full/',
                                    load_emissivity=False)

H I psplups file not found
He II psplups file not found
N VII psplups file not found
O VIII psplups file not found
Ne X psplups file not found
Na XI psplups file not found
Mg XII psplups file not found
Al XIII psplups file not found
Si XIV psplups file not found
P XV psplups file not found
S XVI psplups file not found
Cl XVII psplups file not found
Ar XVIII psplups file not found
K XIX psplups file not found
Ca XX psplups file not found
Fe XXVI psplups file not found
Ni XXVIII psplups file not found
He I psplups file not found
C V psplups file not found
N VI psplups file not found
O VII psplups file not found
Ne IX psplups file not found
Na X psplups file not found
Mg XI psplups file not found
Al XII psplups file not found
Si XIII psplups file not found
P XIV psplups file not found
S XV psplups file not found
Cl XVI psplups file not found
Ar XVII psplups file not found
K XVIII psplups file not found
Ca XIX psplups file not found
Fe XXV psplups file not found
Ni XXVII psplups file not f

Make a frame for each timestep

In [5]:
base_format = '/data/datadrive1/ar_forward_modeling/systematic_ar_study/noaa1109_tn5000/Hinode_EIS/170-210/map_t{:06d}.h5'

In [6]:
def make_eis_frame(time):
    # time, load, wavelength
    i_time = np.where(eis.observing_time==time)[0][0]
    tmp_cube = EISCube(base_format.format(i_time))
    i_wavelength = np.where(np.isclose(tmp_cube.wavelength.value,emiss_model.resolved_wavelengths[8].value,
                                       rtol=0.,atol=1e-2))[0][0]
    # cut and integrate
    tmp_map = tmp_cube[i_wavelength-5:i_wavelength+5].integrated_intensity
    tmp_map.data = gaussian_filter(tmp_map.data,(eis.channels[0]['gaussian_width']['y'].value,
                                                 eis.channels[0]['gaussian_width']['x'].value))
    tmp_map = tmp_map.submap(u.Quantity((250,450),u.arcsec),u.Quantity((150,360),u.arcsec))
    #plot
    fig = plt.figure(figsize=(8,6))
    ax = fig.gca(projection=tmp_map)
    tmp_map.plot(annotate=False,axes=ax,title=False,
                 norm=matplotlib.colors.SymLogNorm(1,vmin=1e2,vmax=2e4))
    ax.coords[0].grid(alpha=0)
    ax.coords[1].grid(alpha=0)
    ax.coords[0].set_ticks(size=5)
    ax.coords[1].set_ticks(size=5)
    ax.set_xlabel(r'$x$ [{}]'.format(u.Unit(tmp_map.meta['cunit1'])))
    ax.set_ylabel(r'$y$ [{}]'.format(u.Unit(tmp_map.meta['cunit2'])))
    xtext,ytext = tmp_map.data_to_pixel(260*u.arcsec,330*u.arcsec) 
    ax.text(xtext.value,ytext.value,r'$t={}$ s'.format(time.value),color='w',fontsize=14)
    ax.set_title(r'Hinode/EIS Fe XII 195 $\mathring{\mathrm{A}}$')
    fig.savefig('movie_frames/frame_{:06d}.pdf'.format(i_time),format='pdf',bbox_inches='tight')
    fig.clf()

In [7]:
for t in np.arange(7500,12500,10)*u.s:
    make_eis_frame(t)

/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot 

In [9]:
%%bash
convert -delay 5 -loop 0 movie_frames/fram*.pdf hinode_eis_fe12_195.gif